<a href="https://colab.research.google.com/github/redrum88/tensorflow/blob/main/08_introduction_to_nlp_in_tensorflow_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP Fundementals in TensorFlow

NLP has the goal of deriving information out of natural language (could be sequences text or speech)

Another common term for NLP is sequence to sequence problems (seq2seq)

## Check for GPU

In [2]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-1470f6fb-3a84-d005-9017-f6b8e54d728e)


## Get helper functions

In [3]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-02-28 00:29:31--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-02-28 00:29:31 (78.7 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [4]:
# Import series of helper function for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

## Get a text dataset

The dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of Tweets labelled as disaster or not disaster).

See the original source here: https://www.kaggle.com/competitions/nlp-getting-started/overview


In [6]:
# Unzip data
unzip_data("nlp-getting-started.zip")

## Visualizing a text dataset

To visualize our text samples, we first have to read them in, one way to do so would be Python.

But I prefer to get visual straight away.

So another way to do this is to use pandas...

In [7]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [8]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [9]:
# What does the test dataframe look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [10]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [11]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [12]:
# Let's visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5) # create random indexes not highher
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 0 (not real disaster)
Text:
[Question] Is anybody else having this problem with the '7' circle in Epicentre? http://t.co/dsPWS6hJ8w

---

Target: 0 (not real disaster)
Text:
christie keeps telling me that i need to be electrocuted

---

Target: 0 (not real disaster)
Text:
I'm awful at painting.. why did I agree to do an A3 landscape in bloody oils of all paints ??

---

Target: 1 (real disaster)
Text:
M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ

---

Target: 1 (real disaster)
Text:
Ignition Knock (Detonation) Sensor-Senso Standard KS94 http://t.co/dY1erSDcRh http://t.co/m4cPmxmuRK

---



### Split data into training and validation sets

In [13]:
from sklearn.model_selection import train_test_split


In [14]:
# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [15]:
# Check the lenghts
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [16]:
# Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

## Converting text into numbers

When dealing with a text problem, one of the first things you'll have to do before you can build a model is to convert your text to numbers.

There are a few ways to do this, namely:
* Tokenization - direct mapping of token (a token could be a word or a character) to number
* Embedding - create a matrix of feature vector for each token (the size of the feature vector can be defined and this embedding can be learned)

### Text vectorization (tokenization)

In [43]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=10000, # how many words in the vocabulary (automatically add <OOV>)
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None, # how long do you want your sequences to be? 
                                    pad_to_max_tokens=True)

In [44]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [45]:
# Setup text vectorization variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does a model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [46]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [47]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [48]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}")
print(f"Vectorized version:")
text_vectorizer([random_sentence])

Original text:
 Usama bin Ladins family dead in airplane crash. Naturally no accident.
Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[7132, 1714,    1,  302,  115,    4,  444,   85,    1,   40,  128,
           0,    0,    0,    0]])>

In [49]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() # get all of the unique words in  our training data
top_5_words = words_in_vocab[:5] # get the most common words
bottom_5_words = words_in_vocab[-5:] # get the least common words
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most common words: {top_5_words}")
print(f"5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using an Embedding Layer

To make our embedding, we're going to use TensorFlow's embedding Layer

The parameters we care most about for our embedding layer:
* `input_dim` = the size of our vocabulary
* `output_dim` = the size of the output embedding vector, for example, a value of 100 would mean each token gets represented by a vector 100 long
* `input_length` = length of the sequences being passed to the embedding layer


In [50]:
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             input_length=max_length)

embedding

In [51]:
# Get a random sentence from the training set
random_sentence =  random.choice(train_sentences)
print(f"Original text:\n {random_sentence}")
print(f"Embedded version:")

# Embed the random sentence (turn it into dense vectors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 Learning from the Legacy of a Catastrophic Eruption http://t.co/RbmuCURS2F
Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.04820633, -0.04454229, -0.00013313, ...,  0.0155263 ,
         -0.02196758,  0.01260853],
        [ 0.02151294, -0.00519247,  0.01369737, ...,  0.0203027 ,
         -0.03158297,  0.03830179],
        [ 0.04369999,  0.03749937, -0.0365851 , ..., -0.01029367,
         -0.00096049, -0.03262101],
        ...,
        [ 0.00867263, -0.00813615, -0.01741796, ...,  0.00078404,
         -0.04042611,  0.01377967],
        [ 0.00867263, -0.00813615, -0.01741796, ...,  0.00078404,
         -0.04042611,  0.01377967],
        [ 0.00867263, -0.00813615, -0.01741796, ...,  0.00078404,
         -0.04042611,  0.01377967]]], dtype=float32)>

In [52]:
# Check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.04820633, -0.04454229, -0.00013313,  0.03293004,  0.03200927,
        -0.04345265,  0.02404994, -0.0480412 ,  0.04998061,  0.03235222,
         0.01817361, -0.04342909,  0.00551315,  0.02474052,  0.03009141,
         0.00513109, -0.04496335,  0.01911667, -0.00819517, -0.01549089,
        -0.02531493,  0.03904209, -0.02515752,  0.00779252, -0.04183686,
         0.03891369, -0.0302701 , -0.02490915, -0.04391092, -0.04665872,
         0.01917653,  0.02082917,  0.04004467,  0.03728086, -0.02711285,
         0.01261056, -0.04743552, -0.02076852, -0.03112843, -0.00402208,
         0.01275912, -0.01653515,  0.04077942, -0.03719259,  0.02531945,
        -0.02440339, -0.02509256,  0.00730171,  0.00679523,  0.01741211,
         0.03171548,  0.00343505,  0.02406833,  0.02169875,  0.02208854,
        -0.04594969, -0.01460892, -0.03321331, -0.02625862,  0.01389534,
         0.00197981, -0.00824392, -0.04199411,  0.00678479, -0.02317923,
  

## Modelling a text dataset (running a series of experiments)

Now we've got a way to turn our text sequences into numbers,
it's time to start building a series of modelling experiments.

We'll start with a baseline and move on from there.

* Model 0: Naive bayes (baseline)
* Model 1: Feed-forward neural network (dense model)
* Model 2: LSTM model (RNN)
* Model 3: GRU model (RNN)
* Model 4: Bidirection-LSTM model (RNN)
* Model 5: 1D Convolutional Neural Network (CNN)
* Model 6: TensorFlow Hub Pretrained Feature Extractor (using transfer learning for NLP)
* Model 7: Same as model 6 with 10% of training data

How are we going to approach all of these?

Use the standard steps in modelling with TensorFlow:

* Create a model
* Build a model
* Fit a model
* Evaluate our model

### Model 0: Getting a baseline

As with all machine learning modelling experiments, it's important to create a baseline model so you've got a benchmark for future experiments to build upon.

To create our baseline, we'll use Sklearn's Multinomial Naive Bayes using the TF-IDF formula to convert our words to numbers.

>🔑**Note:** It's common practice to use non-DL algorithms as a baseline because of their speed and then later using DL to see if you can improve upon them.

In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [54]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [55]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

from pandas.core.arrays.timedeltas import precision_from_unit
### Creating an evaluation function for our model experiments

We could evaluate all of our model's predictions with different metrics every time, however, this will be cumbersome and could work with a function...

Let's create one to compare our model's predictions with the truth labels using the following metrics:
* Accuracy
* Precision
* Recall
* F1-score

For a deep overview of many different evaluaten methods, see the Sklearn documentation


In [56]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1-score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1": model_f1}
  return model_results

In [57]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 1: A simple dense model

In [58]:
# Create a tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create a directory to save TensorBoard logs
SAVE_DIR = "model_logs"


In [59]:
# build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string) # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numberized inputs
x = layers.GlobalAveragePooling1D()(x) # condense the feature vector for each token to one vector
outputs = layers.Dense(1, activation="sigmoid")(x) # Create the output layer,
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [60]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [61]:
# Compile model
model_1.compile(loss=tf.keras.losses.binary_crossentropy,
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [62]:
# Fit the model
model_1_history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20230228-003154
Epoch 1/5
215/215 [==============================] - 16s 53ms/step - loss: 0.6099 - accuracy: 0.6983 - val_loss: 0.5347 - val_accuracy: 0.7612
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.4399 - accuracy: 0.8197 - val_loss: 0.4736 - val_accuracy: 0.7835
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.3455 - accuracy: 0.8602 - val_loss: 0.4601 - val_accuracy: 0.7887
Epoch 4/5
215/215 [==============================] - 1s 7ms/step - loss: 0.2829 - accuracy: 0.8907 - val_loss: 0.4721 - val_accuracy: 0.7940
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.2358 - accuracy: 0.9130 - val_loss: 0.4775 - val_accuracy: 0.7835


In [63]:
# Check the results
model_1.evaluate(val_sentences, val_labels)


24/24 [==============================] - 0s 3ms/step - loss: 0.4775 - accuracy: 0.7835


[0.47747474908828735, 0.7834645509719849]

In [64]:
# Make some predictions and evaluate those
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 [==============================] - 0s 2ms/step


(762, 1)

In [65]:
model_1_pred_probs[0]

array([0.4271937], dtype=float32)

In [66]:
# Convert model prediction probabilities to label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [67]:
# Calculate our model_1 results
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 78.34645669291339,
 'precision': 0.7868445599717488,
 'recall': 0.7834645669291339,
 'f1': 0.7809185675137833}

In [68]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

In [69]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

## Visualizing learned embeddings

In [70]:
# Get the vocavulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [71]:
max_vocab_length

10000

In [72]:
# Model 1 summary
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [73]:
# Get the weight matrix of embedding layer
# (these are the numerical representations of each token in our training data, which have been learned for 5 epochs)
embed_weights = model_1.get_layer("embedding").get_weights()[0]
embed_weights

array([[-0.00604614, -0.02455791, -0.0336865 , ..., -0.01590004,
        -0.05731817,  0.0302482 ],
       [ 0.02005302, -0.05532179,  0.03080012, ...,  0.00511222,
        -0.0398998 ,  0.02319344],
       [ 0.03045174,  0.02214958, -0.05258747, ..., -0.02650944,
        -0.02006681, -0.0168012 ],
       ...,
       [-0.04247764, -0.00454251, -0.03103136, ..., -0.02370731,
         0.04350227,  0.01461304],
       [-0.03320165, -0.00933719, -0.01617314, ..., -0.06089494,
        -0.01337566,  0.05641355],
       [-0.0214838 , -0.06737967, -0.09886493, ..., -0.06565437,
        -0.07382254,  0.01341268]], dtype=float32)

Now we've got the embedding matrix our model has learned to represent our tokens, let's see how we can visualize it.

To do so, TensorFlow has a handy tool called projector: https://projector.tensorflow.org/

And TensorFlow also has an increadible guide on word embeddings themselves: https://www.tensorflow.org/text/guide/word_embeddings

In [74]:
embed_weights.shape

(10000, 128)

In [75]:
# Create embedding files (we got this from TensorFlow's word embeddings documentation)
import io
out_v = io.open("vectors.tsv", "w", encoding="utf-8")
out_m = io.open("metadata.tsv", "w", encoding="utf-8")

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [76]:
# Download files from Colab to upload to projector
try:
  from google.colab import files
  files.download("vectors.tsv")
  files.download("metadata.tsv")
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Downloading files above we can visualize them using http://projector.tensorflow.org and  click the "load" button on the left hand side.

## Recurrent Neural Networks (RNN's)

RNN's are useful for sequence data.

The premise of a recurrent neural network is to use the representation of a previous input to aid the representation of a later input.

> **Resources:** If you want an overview of the internals of a recurrent neural network, see the following:
- MIT's sequence modelling lecture https://www.youtube.com/watch?v=qjrad0V0uJE
- Chris Olah's intro to LSTM: https://colah.github.io/posts/2015-08-Understanding-LSTMs/
- Andrej Karpathy's the unreasonable effectiveness of RNN's: http://karpathy.github.io/2015/05/21/rnn-effectiveness/

### Model 2: LSTM

LSTM = long short term memory (one of the most popular LSTM cells)

Our structure of an RNN typically looks like this:

```
Input (text) -> Tokenize -> Embedding -> Layers (RNNs/dense) -> Output (label probability)
```

In [77]:
# Create an LSTM model
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
# print(x.shape)
# x = layers.LSTM(units=64, return_sequences=True)(x)
# print(x.shape)
x = layers.LSTM(64)(x)
# print(x.shape)
# x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")


In [78]:
# Get a summary
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [79]:
# Compile the model
model_2.compile(loss=tf.keras.losses.binary_crossentropy,
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [80]:
# Fit the model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20230228-003238
Epoch 1/5
215/215 [==============================] - 13s 42ms/step - loss: 0.2280 - accuracy: 0.9167 - val_loss: 0.4889 - val_accuracy: 0.7822
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.1561 - accuracy: 0.9445 - val_loss: 0.6289 - val_accuracy: 0.7887
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.1303 - accuracy: 0.9508 - val_loss: 0.6934 - val_accuracy: 0.7835
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.1090 - accuracy: 0.9587 - val_loss: 0.7435 - val_accuracy: 0.7795
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0842 - accuracy: 0.9680 - val_loss: 0.7656 - val_accuracy: 0.7808


In [81]:
# Make predictions with LSTM model
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 0s 3ms/step


array([[0.01449572],
       [0.84713936],
       [0.9997837 ],
       [0.08885442],
       [0.00255558],
       [0.99932015],
       [0.61607265],
       [0.9998479 ],
       [0.9997569 ],
       [0.26291513]], dtype=float32)

In [82]:
# Convert model 2 pred probs to labels
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [83]:
model_2_results = calculate_results(val_labels, model_2_preds)
model_2_results

{'accuracy': 78.08398950131233,
 'precision': 0.7870588757050528,
 'recall': 0.7808398950131233,
 'f1': 0.77721008098132}

from types import UnionType
### Model 3: GRU

Another popular and effective RNN component is the GRU or gated recurrent unit.

The GRU cell has similar features to an LSTM cell but has less parameters.


In [84]:
# Build an RNN using the GRU cell
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64, return_sequences=True)(x) # if you want to stack recurrent layers on top of each other, you need return_sequences=True
x = layers.LSTM(64, return_sequences=True)(x)
x = layers.GRU(64)(x)
x = layers.Dense(64, activation="relu")(x)
# x = layers.GlobalAveragePooling1D()(x)
ouputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, ouputs, name="model_3_GRU")

In [85]:
# Get a summary
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 15, 64)            37248     
                                                                 
 lstm_1 (LSTM)               (None, 15, 64)            33024     
                                                                 
 gru_1 (GRU)                 (None, 64)                24960     
                                                       

In [86]:
# Compile the model
model_3.compile(loss=tf.keras.losses.binary_crossentropy,
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Fit the model
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, experiment_name="model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20230228-003302
Epoch 1/5
215/215 [==============================] - 16s 48ms/step - loss: 0.1377 - accuracy: 0.9562 - val_loss: 0.9571 - val_accuracy: 0.7677
Epoch 2/5
215/215 [==============================] - 4s 19ms/step - loss: 0.0782 - accuracy: 0.9689 - val_loss: 0.9098 - val_accuracy: 0.7664
Epoch 3/5
215/215 [==============================] - 4s 16ms/step - loss: 0.0698 - accuracy: 0.9723 - val_loss: 1.1554 - val_accuracy: 0.7730
Epoch 4/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0549 - accuracy: 0.9755 - val_loss: 0.9832 - val_accuracy: 0.7795
Epoch 5/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0545 - accuracy: 0.9739 - val_loss: 1.3824 - val_accuracy: 0.7717


In [87]:
# Make some predictions withour GRU model
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

24/24 [==============================] - 1s 4ms/step


array([[1.5389991e-02],
       [9.1427785e-01],
       [9.9996889e-01],
       [5.9674766e-02],
       [7.0517854e-05],
       [9.9959296e-01],
       [9.8520792e-01],
       [9.9996829e-01],
       [9.9996114e-01],
       [9.9768090e-01]], dtype=float32)

In [88]:
# Convert model 3 pred probs to labels
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [89]:
# Calculate model 3 results
model_3_results = calculate_results(val_labels, model_3_preds)
model_3_results

{'accuracy': 77.16535433070865,
 'precision': 0.7715668446582516,
 'recall': 0.7716535433070866,
 'f1': 0.7706989403422689}

### Model 4: Bidirectional RNN

Normal RNN's go from left to right (just like you'd read an English sentence) however, a bidirectional RNN goes from right to left as well as left to right.

In [90]:
# Build a bidirectional RNN in TensorFlow
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.GRU(64))(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name="model_4_bidirectional")

In [91]:
# Get a summary
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 15, 128)          98816     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              74496     
 nal)                                                            
                                             

In [92]:
# Compile the model
model_4.compile(loss=tf.keras.losses.binary_crossentropy,
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Fit the model
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, experiment_name="model_4_bidirectional")])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20230228-010156
Epoch 1/5
215/215 [==============================] - 18s 47ms/step - loss: 0.1098 - accuracy: 0.9634 - val_loss: 0.7428 - val_accuracy: 0.7690
Epoch 2/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0571 - accuracy: 0.9752 - val_loss: 1.0648 - val_accuracy: 0.7743
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0460 - accuracy: 0.9771 - val_loss: 1.4026 - val_accuracy: 0.7677
Epoch 4/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0442 - accuracy: 0.9781 - val_loss: 1.2934 - val_accuracy: 0.7717
Epoch 5/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0426 - accuracy: 0.9788 - val_loss: 1.2896 - val_accuracy: 0.7585


In [93]:
# Make predictions with our bidirectional model
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

24/24 [==============================] - 2s 7ms/step


array([[1.3812417e-01],
       [7.8952008e-01],
       [9.9992657e-01],
       [5.2796841e-01],
       [3.3894368e-05],
       [9.9981076e-01],
       [9.2457980e-01],
       [9.9996567e-01],
       [9.9991846e-01],
       [9.9768746e-01]], dtype=float32)

In [94]:
# Convert pred probs to pred labels
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 1., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [95]:
# Calculate the results of our bidirectional model
model_4_results = calculate_results(val_labels, model_4_preds)
model_4_results

{'accuracy': 75.8530183727034,
 'precision': 0.7581545555800725,
 'recall': 0.7585301837270341,
 'f1': 0.7582109503371647}

## Convolution Neural Networks for Text (and other types of sequences)

We've used CNNs for images but images are typically 2D (height x width)...  however, our text data is 1D.

Previously we've `Conv2D` for our image data but now we're going to use `Conv1D`.

The typical structure of a Conv1D model for sequences (in our case, text):

```
Inputs (text) -> Tokenization -> Embedding -> Layers (typically Conv1D + pooling) -> Outputs (class probabilities)
```

### Model 5: Conv1D

In [97]:
# Test out our embedding layer, Conv1D layer and max pooling
embedding_test = embedding(text_vectorizer(["this is a test sentence"])) # turn target sequence into enbedding
conv_1d = layers.Conv1D(filters=32,
                        kernel_size=5, # this is also referred to as an ngram of 5 (meaning it looks at 5 words at time)
                        activation="relu",
                        padding="valid")
conv_1d_output = conv_1d(embedding_test) # pass test embedding through conv1d layer
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output) # equivalent to "get the most important feature" or "get the feature with the highest value"

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [98]:
embedding_test

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.01837275, -0.01023178,  0.02904946, ..., -0.01039229,
         -0.02253255,  0.0040664 ],
        [ 0.00893986, -0.05644998, -0.04498443, ...,  0.00022852,
          0.01624105, -0.04684821],
        [ 0.02263116, -0.03012851, -0.0690889 , ..., -0.00570576,
          0.00414699, -0.01312734],
        ...,
        [-0.00404348, -0.00832877, -0.0254778 , ..., -0.00338171,
         -0.03694992,  0.01543201],
        [-0.00404348, -0.00832877, -0.0254778 , ..., -0.00338171,
         -0.03694992,  0.01543201],
        [-0.00404348, -0.00832877, -0.0254778 , ..., -0.00338171,
         -0.03694992,  0.01543201]]], dtype=float32)>

In [99]:
conv_1d_output

<tf.Tensor: shape=(1, 11, 32), dtype=float32, numpy=
array([[[0.        , 0.0586189 , 0.        , 0.07847391, 0.05962238,
         0.        , 0.05856267, 0.        , 0.06214723, 0.        ,
         0.        , 0.        , 0.07392804, 0.07645431, 0.06094851,
         0.        , 0.03548915, 0.        , 0.00634056, 0.03986619,
         0.        , 0.06981476, 0.        , 0.07324505, 0.0473678 ,
         0.        , 0.        , 0.04989267, 0.        , 0.        ,
         0.03933841, 0.01047022],
        [0.02759024, 0.02608985, 0.00634827, 0.04357117, 0.        ,
         0.02616687, 0.01149545, 0.0405029 , 0.        , 0.05068357,
         0.01555487, 0.02605198, 0.01894793, 0.        , 0.        ,
         0.        , 0.02217261, 0.02321361, 0.        , 0.        ,
         0.01220655, 0.        , 0.        , 0.00372911, 0.        ,
         0.04757422, 0.07088938, 0.        , 0.        , 0.        ,
         0.02510871, 0.02368614],
        [0.        , 0.        , 0.        , 0.    

In [104]:
# Build model 5
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters=64,
                  kernel_size=5,
                  strides=1,
                  padding="valid",
                  activation="relu")(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_5 = tf.keras.Model(inputs, outputs, name="model_5_conv1d")

In [105]:
# Get a summary
model_5.summary()

Model: "model_5_conv1d"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_4 (Conv1D)           (None, 11, 64)            41024     
                                                                 
 global_max_pooling1d_4 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_7 (Dense)             (None, 1)              

In [106]:
# Compile the model
model_5.compile(loss=tf.keras.losses.binary_crossentropy,
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Fit the model
model_5_history = model_5.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     experiment_name="model_5_conv1d")])

Saving TensorBoard log files to: model_logs/model_5_conv1d/20230228-015517
Epoch 1/5
215/215 [==============================] - 10s 40ms/step - loss: 0.0925 - accuracy: 0.9746 - val_loss: 1.0326 - val_accuracy: 0.7677
Epoch 2/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0542 - accuracy: 0.9777 - val_loss: 1.1217 - val_accuracy: 0.7625
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0500 - accuracy: 0.9793 - val_loss: 1.1792 - val_accuracy: 0.7598
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0472 - accuracy: 0.9806 - val_loss: 1.2669 - val_accuracy: 0.7651
Epoch 5/5
215/215 [==============================] - 1s 7ms/step - loss: 0.0457 - accuracy: 0.9810 - val_loss: 1.2122 - val_accuracy: 0.7520


In [107]:
# Make some preditions with our Conv1D model
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:10]

24/24 [==============================] - 0s 4ms/step


array([[1.50451213e-01],
       [6.38622820e-01],
       [9.99946117e-01],
       [1.14419274e-01],
       [3.49078221e-07],
       [9.97060955e-01],
       [8.05300295e-01],
       [9.99994397e-01],
       [9.99999523e-01],
       [9.41084266e-01]], dtype=float32)

In [108]:
# Convert  model 5 pred probs to labels
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [109]:
# Evaluate model 5 predictions
model_5_results = calculate_results(val_labels, model_5_preds)
model_5_results

{'accuracy': 75.19685039370079,
 'precision': 0.751515348822761,
 'recall': 0.7519685039370079,
 'f1': 0.7512699382003619}